# **Workshop Demo**: Introduction to Machine Learning in Finance

- **Objectives**


# 1. Price Forecasting

In [ ]:
# Hassan TODO
print("hello world")

# 2. Risk Forecasting

In [ ]:
# Nico TODO
print("hello")

# 3. Deep Learning in Option Pricing

**What is Deep Learning**

- [Deep Learning](https://www.ibm.com/topics/deep-learning) is a subset of machine learning that uses multilayered neural networks to simulate the complex decision-making power of the human brain.

- Main difference between deep learning and machine learning is the structure of the underlying neural network architecture. "Nondeep" traditional ML models use simple neural networks with one or two computational layers. Deep learning models use neural networks with one or two compuetational layers. Deep learning models use three or more layers (typically hundreds or thousands of layers) to train the models.

- Consists of multiple layers of interconnected notes, each building on previous to refine and optimize the prediction -> forward propogation. The `IN` and `OUT` layers of a deep neural network are called **visible** layers

- Another process, backward propagation, is used to calculate errors in predictions and then adjust the weights and biases of between the node layers accordingly. The common approach to do this is via `gradient descent`.

- Common deep learning neural networks: 
    - **CNNs** (convolution neural networks): computer vision and image classification applications.
    - **RNNs** (recurrent neural networks): natural language and speech recognition.

**What is Option Pricing**
- Formal definition of an [option](https://corporatefinanceinstitute.com/resources/derivatives/option-pricing-models/) states that it is a type of contract between two parties that provides one party the right, but **not** the obligation, to buy or sell the underlying asset at a predetermined price before or at expiration day.

    - Call option = buying
    - Put option = selling

- Buying or selling an option comes with a price called the option's premium. Buyer of an option pay the premium and sellers receive the premium

- Some common factors determining the value of an option:

    - [Intrinsic value](https://www.investopedia.com/terms/i/intrinsicvalue.asp)
    - Time to expiration
    - Volatility
    - Interest rates

(ie. price of a stock rises -> likely price of call option will rise and the price of a put option will fall, vice versa)

- Example pricing models: 

    - Binomial Model
    - Trinomial Model
    - Black-Scholes Model
    - Monte-Carlo Simulation
    

## Black-Scholes

- One of the most commonly used formula to price call and put options, there are many variations of this formula.
- Model was first discovered in 1973 by economists Fischer black and Myron Scholes, developed mainly for pricing European options on stocks.

- Some assumptions:
    - Stock Price Distribution : continuously compounded returns on the stock are normally distributed and independent over time, volatility of continuously compounded returns is known and constant, future dividends are known (as a dollar amount/fixed dividend yield)
    - Economic Environment : risk-free rate is known and constant, no transaction costs or taxes, possible to short-sell with no cost and to borrow at the risk-free rate

- An example of deep learning model using Black-Scholes to predict option pricing: [Sanskar](https://github.com/Sanskar02/OPTION_PRICING_MODEL)
- Another example model can be found [here](https://srdas.github.io/Papers/BlackScholesNN.pdf)

# Example Deep Learning Model in Option Pricing

**1. Load and prepare dataset**

- Historical option data from [AlphaVantage](https://www.alphavantage.co/)
- Measures and variables chosen based off factors of Black-Scholes model
- Extracting data for call options only
- Randomly generating interest rate values

In [ ]:
import pandas as pd
from datetime import datetime

# Load the CSV file
file_path = 'C:/Users/ziyin/OneDrive/Desktop/intro-to-ml/historical_options_IBM.csv'
data = pd.read_csv(file_path)

# Convert 'expiration' and 'date' columns to datetime format
data['expiration'] = pd.to_datetime(data['expiration'])
data['current'] = pd.to_datetime(data['date'])

# Calculate time to expiration (T) in years
today = datetime.now().date()
data['T'] = (data['expiration'].dt.date - data['current'].dt.date).apply(lambda x: x.days) / 365

filtered_data = data[data['type']=='call']

# Map relevant columns to the desired format, and add placeholders if necessary
df_formatted = filtered_data.rename(columns={
    'last': 'S',                # Use 'last' as a placeholder for spot price if applicable
    'strike': 'K',
    'implied_volatility': 'sigma'
})

# Add placeholders and 'r' (risk-free rate)
df_formatted['r'] = 0.01  # Placeholder for a standard risk-free rate


# Create a new DataFrame that only contains the desired columns
df = df_formatted[['S', 'K', 'T', 'r', 'sigma']]


**Creating DataFrame**

In [ ]:
df.head(50)

In [ ]:
# Retrieving call values from spreadsheet
df['C'] = filtered_data['bid']
df.head(50)

In [ ]:
# Creating new DataFrame object df1 that points to same objects as df
df1 = df.copy()

In [ ]:
# Normalising the roperty of linear homogeneity in the Black-Scholes model
# Model's output price scales linearly with scale of input
df1["S"] = df1["S"]/df1["K"]
df1["C"] = df1["C"]/df1["K"]

# Now database S represents ratio of spot price to strike price (S/K)
# C represents ratio of normalizes option price (C/K)
df1.head()

In [ ]:
# Removing column K as it is now implied in the dataset under columns S and C
df1.drop(columns = ['K'], inplace = True)
df1.head(50)

In [ ]:
X = df1.copy()
# Dropping columns C and sigma (option price and volatility) to initialse training dataset
X.drop(columns = ['C'], inplace = True)
X.drop(columns = ['sigma'], inplace = True)
X.info()

In [ ]:
# Defining y as normalized C (option price) from before for future reference
y = df1['C']


In [ ]:
# Imports for creation of synthetic datasets and for splitting arrays/DataFrames into random train and test subsets
from sklearn.datasets import make_blobs
from sklearn.model_selection import train_test_split

# Splits feature matrix X and target vector y into training and testing sets
# 80% of data will be used for training and remaining 20% will be used for testing
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state = 1)

# Now stores 80% of X dataframe
X_train

In [ ]:
# Stores 80% of y dataframe, ground truth labels that the model will learn to predict from data in X_train
y_train

In [ ]:
X_test


In [ ]:
y_test

In [ ]:

print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

[TensorFlow](https://www.tensorflow.org/) framework is used for creating and deploying ML models. 

In [ ]:
# importing TensorFlow library to prepare custom activation functions
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, LeakyReLU
from tensorflow.keras import backend as K

# Defining a custom activation function
def custom_activation(x):
    return K.exp(x)


**Building the ANN(artificial neural network) Model**

Three activation layers are added:

- LeakyRELU, function is h(x) = x if x ≥ 0, else h(x) = αx
- ELU (exponential linear unit). The function is h(x) = x if x ≥ 0, else h(x) = α(exp(x)-1).
- ReLU (rectified linear unit). THe function is h(x) = max(x, 0). The gradient of the function is zero in the region where x is negative, and the neuron is not active.

There is many more types of activation functions that can be added to a neural network. 

In [ ]:
nodes = 120
model = Sequential()

# Initial layer
# Adding LeakyRELU activation function
model.add(Dense(nodes, input_dim=X_train.shape[1]))
model.add(LeakyReLU())
# randomly deactivates 25% of nodes during training
model.add(Dropout(0.25))

# Hidden layers
model.add(Dense(nodes, activation='elu'))
model.add(Dropout(0.15))

model.add(Dense(nodes, activation='relu'))
model.add(Dropout(0.35))

model.add(Dense(nodes, activation='elu'))
model.add(Dropout(0.25))

# Output layer
model.add(Dense(1))
model.add(Activation(custom_activation))
          
# Compiles using mean squared error as loss function
# rmspropr optimizer, adapts learning rate during training based on magnitude of recent gradients
model.compile(loss='mse',optimizer='rmsprop')

In [ ]:
# Training the neural network model
# Model is learning from training dataset X_train as input and y_train as target output
# Data is split into mini-batches of 64 samples, updating node wirghts after processing each batch
# Will iterate over entire training dataset 20 times
# 10% of training data is held out as validation data for each epoch
model.fit(X_train, y_train, batch_size=64, epochs=20, validation_split=0.1, verbose=2)

In [ ]:
# Provides summary of neural network model's architecure:
    # Layer type and details, output shape parameters
model.summary()

In [ ]:
import math 
from sklearn.metrics import *
import matplotlib as mpl
import matplotlib.pyplot as plt

# Assess the accuracy of predictions generated by the model, comparing them to actual values
# y is the actual value and y_hat is the predicted
def CheckAccuracy(y,y_hat):
    stats = dict()

    stats['diff'] = y - y_hat
    
    #figure(figsize=(14,10))
    plt.hist(stats['diff'], bins=50,edgecolor='black',color='white')
    plt.xlabel('Diff')
    plt.ylabel('Density')
    plt.show()
    
    return stats

In [ ]:
# Predicted values from the training set
y_train_hat = model.predict(X_train)
#reduce dim (240000,1) -> (240000,) to match y_train's dim
y_train_hat = np.squeeze(y_train_hat)
CheckAccuracy(y_train, y_train_hat)